**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/u8u7vcwy3sosbar/titanic.zip?dl=1",
                            directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Optimalizácia hyperparametrov pomocou Grid Search

V ďalšom notebook-u si ešte pre porovnanie ukážeme ďalšiu jednoduchú metódu optimalizácie hyperparametrov: nazýva sa grid search (mriežkového prehľadávania).

### Načítanie dát a predspracovanie

Načítanie a predspracovanie dát sa zhoduje s tým z predchádzajúceho notebook-u.



In [ ]:
#@title -- Loading and preprocessing: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

# we split the columns into categorical and numeric inputs and the output
categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']
output = ["Survived"]

# we create our preprocessing pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder(categories='auto')),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# we fit the pipeline on the train set and then apply it to both train and test
X_train = input_preproc.fit_transform(df_train[categorical_inputs + numeric_inputs])
Y_train = df_train[output]

X_test = input_preproc.transform(df_test[categorical_inputs + numeric_inputs])
Y_test = df_test[output]

### Grid Search

Pri použití metódy grid search sa definuje "mriežka" všetkých hyperparametrov a vzniknutý priestor sa následne prehľadáva – systematicky a úplne.

Ďalej si môžeme znovu zobraziť dokumentáciu ku triede `DecisionTreeClassifier`, aby sme si pripomenuli, aké hyperparametre bude treba ladiť:



In [ ]:
?DecisionTreeClassifier

---
### Úloha 1: Konfigurácia prehľadávaného priestoru

**V nasledujúcej bunke definujte prehľadávaný priestor `space` pre hyperparametre rozhodovacieho stromu.** 

---
Znovu budeme potrebovať konfigurovať prehľadávaný priestor. V prípade grid search musí byť priestor diskrétny a podľa možnosti pomerne malý, keďže budeme vyčerpávajúcim spôsobom testovať všetky možné konfigurácie.

Budeme používať metódu `GridSearchCV` z balíčka `sklearn`. Pre ňu sa prehľadávaný priestor definuje ako slovník, kde kľúče sú názvy hyperparametrov a hodnoty sú zoznamy možných hodnôt hyperparametrov, napr.:

```
space = {
    # kategorická premenná:
    'cat_var': ["opt1", "opt2", "opt3"],

    # číselná premenná: diskretizujeme
    'num_var': [0.1, 0.5, 1.0]
}
```


In [ ]:
grid = {


    # ---

    
}

### Spustenie optimalizácie

Ďalej môžeme spustiť optimalizáciu – pomocou metódy `GridSearchCV`.



In [ ]:
start = time.time()

model = DecisionTreeClassifier()
grid_search = GridSearchCV(model, grid, n_jobs=-1, cv=10,
                           scoring='f1_macro', verbose=True)
grid_search.fit(X_train, Y_train)

end = time.time()
print(end - start)

Extrahujeme najlepšie parametre:



In [ ]:
best_params = grid_search.best_params_
best_params

### Tréning modelu s najlepšími parametrami

Keď sme identifikovali najlepšie parametre, použijeme ich teraz, aby sme natrénovali nový model: tento raz už s použitím celej tréningovej množiny:



In [ ]:
model = DecisionTreeClassifier(**best_params)
model = model.fit(X_train, Y_train)

### Testovanie

Na záver si model otestujeme na testovacích dátach. Zobrazíme si maticu zámen a naše štandardné metriky.



In [ ]:
y_test = model.predict(X_test)

In [ ]:
cm = pd.crosstab(Y_test.values.reshape(-1), y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm)

In [ ]:
print("Accuracy = {}".format(accuracy_score(Y_test, y_test)))
print("Precision = {}".format(precision_score(Y_test, y_test)))
print("Recall = {}".format(recall_score(Y_test, y_test)))